# Continuous-Time Markov Chains: The Yule-Furry Process

In this notebook, we will simulate a continuous-time Markov chain (CTMC) and 
compare the results to known analytical formulas. The example we will use is 
the **Yule-Furry process**, also known as the linear birth process.

Recall from the board that in this process, each individual independently gives 
birth at rate $\beta$. When the population size is $n$, the total birth rate is 
$n\beta$. Since there are no deaths, the population can only grow.

## Why are inter-event times exponentially distributed?

The defining property of a CTMC is that it is *memoryless*: the future depends 
only on the current state, not on how long the system has been in that state. 
The exponential distribution is the unique continuous distribution with this 
memoryless property â€” if $T \sim \text{Exp}(\lambda)$, then 
$P(T > t + s \mid T > t) = P(T > s)$ for all $t, s \geq 0$.

For a single individual with birth rate $\beta$, the time until its next birth 
is $\text{Exp}(\beta)$. When there are $n$ independent individuals each with 
rate $\beta$, the time until the *first* birth among all of them is the minimum 
of $n$ independent exponentials. A standard result from probability is that the 
minimum of $n$ independent $\text{Exp}(\beta)$ random variables is 
$\text{Exp}(n\beta)$. So when the population is $n$, the time until the next 
event is exponentially distributed with rate $n\beta$ (equivalently, mean 
$1/(n\beta)$).

## The Gillespie algorithm

The **Gillespie algorithm** (also called the stochastic simulation algorithm) is 
a general method for exactly simulating CTMCs. The idea is simple: rather than 
advancing time in fixed steps and checking whether events occur (which would 
require very small time steps for accuracy), we jump directly from one event to 
the next. At each step:
1. Compute the total rate of all possible events given the current state.
2. Draw the time until the next event from the corresponding exponential 
distribution.
3. Determine which event occurs (in our case there is only one type â€” a birth).
4. Update the state and repeat.

This is exact (no discretization error) and efficient, since we skip over all 
the dead time between events. For the Yule-Furry process, the algorithm is 
particularly simple because there is only one type of event (birth), so step 3 
is trivial.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb # computes binomial coefficients, C(n, k)

## Simulating a single realization

The simulation algorithm is as follows:
1. Start with $n_0$ individuals at time $t=0$.
2. Draw an inter-event time $\Delta t$ from $\text{Exp}(n\beta)$. In numpy, 
`rng.exponential(scale)` takes the *scale* parameter, which is $1/\text{rate} = 1/(n\beta)$.
3. Advance time: $t \rightarrow t + \Delta t$.
4. If $t > T_{\max}$, stop.
5. Otherwise, increment the population: $n \rightarrow n + 1$.
6. Record the new time and population size, and go back to step 2.

In the cell below, implement this algorithm using a while loop. Store the 
times and population sizes in lists so that you can plot them afterward. Use 
`plt.step()` to plot the result as a step function (which is the correct 
visualization for a jump process).

In [ ]:
rng = np.random.default_rng()

# Parameters
beta = 0.1
n0 = 5
T_max = 10

# Initialize lists to store the trajectory
times = [0.0]
pop = [n0]

# Run the simulation
t = 0.0
n = n0
while True:
    # Draw inter-event time
    dt = rng.exponential(1.0 / (n * beta))
    t += dt
    if t > T_max:
        break
    n += 1
    times.append(t)
    pop.append(n)

# Append the final time so the plot extends to T_max
times.append(T_max)
pop.append(n)

# Plot the trajectory as a step function
plt.figure(figsize=(8, 4))
plt.step(times, pop, where='post')
plt.xlabel('Time')
plt.ylabel('Population size N(t)')
plt.title('Single realization of the Yule-Furry process, $\\beta$={}, $n_0$={}'.format(beta, n0))
plt.show()

## Running many simulations

A single realization tells us little about the statistics of the process. To 
understand the distribution of outcomes, we need to run many simulations and 
collect the results.

First, wrap your simulation code from above into a function that takes 
parameters and returns the trajectory (times and population sizes). Then, run 
it `N_sim` times and:
1. Plot all trajectories on the same axes.
2. Overlay the analytical expected value $E[N(t)] = n_0 e^{\beta t}$ as a 
dashed line for comparison.

In [ ]:
def simulate_yule_furry(beta, n0, T_max, rng):
    """Simulate one realization of the Yule-Furry process.
    
    Returns lists of times and population sizes."""
    times = [0.0]
    pop = [n0]
    t = 0.0
    n = n0
    while True:
        dt = rng.exponential(1.0 / (n * beta))
        t += dt
        if t > T_max:
            break
        n += 1
        times.append(t)
        pop.append(n)
    # Extend to T_max
    times.append(T_max)
    pop.append(n)
    return times, pop

# Parameters
beta = 0.1
n0 = 5
T_max = 10
N_sim = 200

rng = np.random.default_rng()

# Run simulations and collect final population sizes
final_pops = np.zeros(N_sim, dtype=int)
plt.figure(figsize=(10, 5))
for i in range(N_sim):
    t_traj, n_traj = simulate_yule_furry(beta, n0, T_max, rng)
    final_pops[i] = n_traj[-1]
    plt.step(t_traj, n_traj, where='post', alpha=0.15, color='blue', linewidth=0.5)

# Overlay the analytical expected value
t_analytical = np.linspace(0, T_max, 200)
E_N = n0 * np.exp(beta * t_analytical)
plt.plot(t_analytical, E_N, 'r--', linewidth=2, label='$E[N(t)] = n_0 e^{\\beta t}$')
plt.xlabel('Time')
plt.ylabel('Population size N(t)')
plt.title('{} realizations of the Yule-Furry process'.format(N_sim))
plt.legend()
plt.show()

print('Sample mean of N(T_max): {:.2f}'.format(np.mean(final_pops)))
print('Analytical E[N(T_max)]:  {:.2f}'.format(n0 * np.exp(beta * T_max)))
print('Sample variance:         {:.2f}'.format(np.var(final_pops)))
print('Analytical variance:     {:.2f}'.format(n0 * (1 - np.exp(-beta*T_max)) * np.exp(2*beta*T_max)))

## Comparing to the analytical distribution

The Yule-Furry process has a known probability distribution. For $n \geq n_0$,
$$
p_n(t) = \binom{n-1}{n_0-1} e^{-\beta n_0 t} \left(1 - e^{-\beta t}\right)^{n - n_0}.
$$
This is a *negative binomial distribution* in which the probability of 
"success" in a single trial, $e^{-\beta t}$, decreases exponentially with time.

In the cell below, create a histogram of the final population sizes from your 
simulations and overlay the analytical PMF. Use `density=True` in `plt.hist()` 
so that the histogram is normalized to a probability distribution. To compute 
the binomial coefficient, use `scipy.special.comb` which we imported above.

In [ ]:
# Compute the analytical PMF over a range of population sizes
n_vals = np.arange(n0, np.max(final_pops) + 5)
p_analytical = comb(n_vals - 1, n0 - 1) * \
    np.exp(-beta * n0 * T_max) * (1 - np.exp(-beta * T_max))**(n_vals - n0)

# Plot histogram of simulation results vs analytical PMF
plt.figure(figsize=(10, 5))
# Use integer bins centered on each value
bins = np.arange(n0 - 0.5, np.max(final_pops) + 1.5, 1)
plt.hist(final_pops, bins=bins, density=True, alpha=0.6, label='Simulation ({} runs)'.format(N_sim))
plt.plot(n_vals, p_analytical, 'ro-', markersize=4, label='Analytical PMF')
plt.xlabel('Population size n')
plt.ylabel('Probability')
plt.title('Distribution of N(T) at T={}, $\\beta$={}, $n_0$={}'.format(T_max, beta, n0))
plt.legend()
plt.show()

## Further exploration

Try experimenting with different parameter values:
- What happens when you increase $\beta$? The population grows faster, leading 
to larger final sizes and wider distributions. Be careful with large $\beta$ or 
large $T_{\max}$ as the population can grow very large and the simulation will 
slow down.
- What happens when $n_0 = 1$? The distribution becomes a geometric 
distribution. Try it!
- How many simulations do you need for the histogram to closely match the 
analytical PMF?

**Even better:** alter the code you have written to be a birth-death process! 
Assume a constant death rate $\mu$ similar to the rate $\beta$. Then the rate at 
which each event occurs is given by $n(\beta+\mu)$. When each event occurs, you 
then have to figure out if it was a birth or a death. It is a birth with probability 
$\frac{\beta}{\beta+\mu}$ and a death with probability $\frac{\mu}{\beta+\mu}$. 
Update the population accordingly and then continue.

## 2D Random Walks

So far in this notebook we have simulated a continuous-time process --- one where events happen at random times. Now we shift to a **discrete-time** *spatial* process: the **2D random walk**. This connects naturally to ideas from the Branching Processes notebook (notebook 4), where we also simulated many independent realizations of a stochastic process and studied the resulting distributions.

A random walk is a sequence of positions in space where each step to a new location is chosen randomly. In the simplest 2D version, a walker starts at the origin $(0, 0)$ and at each time step moves exactly one unit in one of the four cardinal directions --- up, down, left, or right --- each chosen with equal probability $\frac{1}{4}$.

Random walks are foundational models in probability and applied mathematics. They arise in physics (Brownian motion), biology (animal foraging, cell migration), finance (stock prices), and as the basis for more complex spatial stochastic models including CTMCs on grids.

The four possible steps, in $(x, y)$ coordinates, are:
$$
(+1, 0), \quad (-1, 0), \quad (0, +1), \quad (0, -1)
$$

**Hint:** One clean way to represent these four choices is as a numpy array of shape $(4, 2)$, then use `rng.integers` to pick a row at each step.

This exercise is an adaptation of the Chapter 7 lab from your book.

In [ ]:
rng = np.random.default_rng()

# Parameters: number of steps for a single walk
N_steps = 200

# The four possible unit steps: right, left, up, down
steps = np.array([[1, 0], [-1, 0], [0, 1], [0, -1]])

# Simulate a single 2D random walk
pos = np.zeros((N_steps + 1, 2), dtype=int)   # row i holds (x, y) after i steps
for i in range(N_steps):
    pos[i+1] = pos[i] + steps[rng.integers(4)]

# Plot the trajectory
plt.figure(figsize=(6, 6))
plt.plot(pos[:, 0], pos[:, 1], lw=0.8, color='steelblue', zorder=1)
plt.scatter(*pos[0],  color='green', s=60, label='Start (0,0)')
plt.scatter(*pos[-1], color='red', s=60, label=f'End ({int(pos[-1,0])}, {int(pos[-1,1])})')
plt.axis('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'Single 2D random walk ({N_steps} steps)')
plt.legend()
plt.tight_layout()
plt.show()

## Many random walks: scatter plot of final positions

Now run 1,000 random walks (each of `N_steps` steps) and collect the final position of each one.
Make a scatter plot of all final positions. What shape does the cloud of endpoints form?

In [ ]:
N_walks = 1000

# Run N_walks random walks and collect final positions
# Efficient approach: draw all step indices at once as a 2D array
step_indices = rng.integers(0, 4, size=(N_walks, N_steps))  # shape (N_walks, N_steps)
all_steps = steps[step_indices]                              # shape (N_walks, N_steps, 2)
final_positions = np.sum(all_steps, axis=1)                 # shape (N_walks, 2)

# Scatter plot of final positions
plt.figure(figsize=(6, 6))
plt.scatter(final_positions[:, 0], final_positions[:, 1],
            s=5, alpha=0.4, color='steelblue', label='Final positions')
plt.scatter(0, 0, color='red', s=80, label='Origin')
plt.axis('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'Final positions of {N_walks} random walks ({N_steps} steps each)')
plt.legend()
plt.tight_layout()
plt.show()

## Distribution of displacement from the origin

The **displacement** of a random walk is the straight-line distance from the origin to the 
final position: $d = \sqrt{x^2 + y^2}$.

In the cell below, compute the displacement for each of your 1,000 walks and plot a histogram.
What does the distribution look like?

See Chapter 7 lab in your book for context.

In [ ]:
# Compute displacement for each walk
displacement = np.sqrt(final_positions[:, 0]**2 + final_positions[:, 1]**2)

plt.figure(figsize=(7, 4))
plt.hist(displacement, bins=30, density=True, color='steelblue', edgecolor='white', alpha=0.8)
plt.xlabel('Displacement $d = \sqrt{x^2 + y^2}$')
plt.ylabel('Probability density')
plt.title(f'Displacement distribution: {N_walks} walks, {N_steps} steps each')
plt.tight_layout()
plt.show()

print(f'Mean displacement:   {np.mean(displacement):.2f}')
print(f'Std of displacement: {np.std(displacement):.2f}')
# For a 2D random walk of N steps, the displacement has a Rayleigh-like distribution.
# The expected displacement is approximately sqrt(N_steps) for large N_steps.
print(f'sqrt(N_steps):       {N_steps**0.5:.2f}')